## 문제5) 타이타닉 생존자 분류하기 

In [107]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,r2_score,mean_squared_error


In [108]:
import seaborn as sns
df = sns.load_dataset('titanic')
x=df.drop('survived', axis = 1)
y = df['survived']
x_train, x_test , y_train , y_test = train_test_split(x,y,stratify=y, test_size=0.2, random_state=2023)

cust_id = pd.DataFrame(x_test.reset_index())['index']

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

y_train.columns = ['target']
y_test.columns =['target']

In [109]:
print(x_train.info())
print(y_train.info())
print(x_test.info())                   

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 3 to 608
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   pclass       712 non-null    int64   
 1   sex          712 non-null    object  
 2   age          579 non-null    float64 
 3   sibsp        712 non-null    int64   
 4   parch        712 non-null    int64   
 5   fare         712 non-null    float64 
 6   embarked     710 non-null    object  
 7   class        712 non-null    category
 8   who          712 non-null    object  
 9   adult_male   712 non-null    bool    
 10  deck         164 non-null    category
 11  embark_town  710 non-null    object  
 12  alive        712 non-null    object  
 13  alone        712 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(3), object(5)
memory usage: 64.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 3 to 608
Data columns (total 1 columns):
 #   Column  

In [110]:
x_train.describe().T

,count,mean,std,min,25%,50%,75%,max
pclass,712.0,2.307584,0.834926,1.00,2.0000,3.0000,3.000,3.0000
age,579.0,29.479568,14.355304,0.42,20.0000,28.0000,38.000,74.0000
sibsp,712.0,0.518258,1.094522,0.00,0.0000,0.0000,1.000,8.0000
parch,712.0,0.372191,0.792341,0.00,0.0000,0.0000,0.000,6.0000
fare,712.0,31.741836,45.403910,0.00,7.8958,14.4542,31.275,512.3292


In [111]:
x_train.describe(include='category').T

,count,unique,top,freq
class,712,3,Third,391
deck,164,7,C,47


In [112]:
print(x_train.isnull().sum())

pclass           0
sex              0
age            133
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           548
embark_town      2
alive            0
alone            0
dtype: int64


In [113]:
print(x_test.isnull().sum())


pclass           0
sex              0
age             44
sibsp            0
parch            0
fare             0
embarked         0
class            0
who              0
adult_male       0
deck           140
embark_town      0
alive            0
alone            0
dtype: int64


In [114]:
x_train['embarked'].mode()[0]

'S'

In [115]:
# 결측치 컬럼 처리 
# x_train : age, deck, embarked,embark_town, 
# x_test : age, deck 
# 결측치 너무 많은 컬럼은 삭제, 나머지는 적당한 값으로 대체
# mode 값, median 값 미리 출력 해보고 fillna 하자 
x_train.drop(columns='deck', inplace=True)
x_train['age'] = x_train['age'].fillna(x_train['age'].median())
x_train['embarked'] = x_train['embarked'].fillna(x_train['embarked'].mode()[0])
x_train['embark_town'] = x_train['embark_town'].fillna(x_train['embark_town'].mode()[0])

x_test.drop(columns='deck',inplace=True)
x_test['age'] = x_test['age'].fillna(x_test['age'].median())



In [116]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())


pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64


In [117]:
# 중복되는 변수 drop 처리
x_train.drop(columns=['class','embark_town','alive'],axis=1,inplace=True)
x_test.drop(columns=['class','embark_town','alive'],axis=1,inplace=True)


In [118]:
x_train =pd.get_dummies(x_train)
x_train.head()

,pclass,age,sibsp,parch,fare,adult_male,alone,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,who_child,who_man,who_woman
3,1,35.0,1,0,53.10,False,False,1,0,0,0,1,0,0,1
517,3,28.0,0,0,24.15,True,True,0,1,0,1,0,0,1,0
861,2,21.0,1,0,11.50,True,False,0,1,0,0,1,0,1,0
487,1,58.0,0,0,29.70,True,True,0,1,1,0,0,0,1,0
58,2,5.0,1,2,27.75,False,False,1,0,0,0,1,1,0,0


In [119]:
x_test=pd.get_dummies(x_test)
x_test.head()

,pclass,age,sibsp,parch,fare,adult_male,alone,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,who_child,who_man,who_woman
800,2,34.0,0,0,13.0000,True,True,0,1,0,0,1,0,1,0
341,1,24.0,3,2,263.0000,False,False,1,0,0,0,1,0,0,1
413,2,29.0,0,0,0.0000,True,True,0,1,0,0,1,0,1,0
575,3,19.0,0,0,14.5000,True,True,0,1,0,0,1,0,1,0
202,3,34.0,0,0,6.4958,True,True,0,1,0,0,1,0,1,0


In [120]:
# 모델 학습 
model = RandomForestClassifier()
model.fit(x_train,y_train['target'])


RandomForestClassifier()

In [121]:
x_train, x_val, y_train,y_val = train_test_split(x_train,y_train['target'],stratify=y_train['target'],test_size=0.2)


In [122]:
y_pred = model.predict(x_val)

In [123]:
accuracy = accuracy_score(y_val,y_pred) # 실제값 , 예측값
f1_s = f1_score(y_val,y_pred,average='macro')
roc_auc = roc_auc_score(y_val,y_pred)

print(accuracy)
print(f1_s)
print(roc_auc)

0.958041958041958
0.9550314465408805
0.9488636363636364


In [125]:
# 결과 제출 
y_result = model.predict(x_test)
result = pd.DataFrame({'cust_id' : cust_id,'result':y_result})
# result.to_csv('result.csv',index=False)
result

,cust_id,result
0,800,0
1,341,1
2,413,0
3,575,0
4,202,0
...,...,...
174,402,0
175,70,0
176,53,1
177,743,0
